### This notebook consist of code for creating the html files for the website each time data is updated.

##### Set-up

In [455]:
# Importing classes
import json
import os
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path

In [456]:
# Defining paths
GROUP_DATA_DIR = Path("/group-data")
MEMBERS_DIR_PATH = GROUP_DATA_DIR / "members/"
WEBSITE_DATA_PATH = GROUP_DATA_DIR / "website_data/"
CONTENT_DIR_PATH = WEBSITE_DATA_PATH / "content"
TEMPLATE_DIR_PATH = GROUP_DATA_DIR.parent / "groupwebsite_generator" / "templates"
HOSTING_PATH = GROUP_DATA_DIR.parent / "kerzendorf-group.github.io"

In [457]:
# Function to create proper HTML file names by replacing spaces with underscores
def page_link(a):
    return a.replace(" ", "_") if " " in a else a

In [458]:
# Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates
environment = Environment(
    loader=FileSystemLoader(TEMPLATE_DIR_PATH), extensions=["jinja2.ext.loopcontrols"]
)
environment.globals["page_link"] = page_link

In [459]:
json_files = [
    "general",
    "homepage",
    "research",
    "support",
    "contact",
]  # List of JSON files to be processed

data = {}
# Looping through JSON files and loading their content into the 'data' dictionary
for json_file in json_files:
    json_file_path = WEBSITE_DATA_PATH / f"{json_file}.json"

    try:
        with open(json_file_path, "r") as json_var:
            data[json_file] = json.load(json_var)
    except (FileNotFoundError, json.JSONDecodeError):
        pass

Creating dataframes for articles which can be updated further 

In [460]:
def load_content_from_files(columns):
    content_data = {col: [] for col in columns}

    for json_file in os.listdir(CONTENT_DIR_PATH):
        if json_file.endswith(".json"):
            json_path = os.path.join(CONTENT_DIR_PATH, json_file)
            with open(json_path, "r") as file:
                info = json.load(file)
                if info.get("display"):
                    for col in columns:
                        content_data[col].append(info.get(col))

    content_df = pd.DataFrame(content_data)
    content_df["date"] = pd.to_datetime(content_df["date"], format="%m-%d-%Y")
    return content_df


def get_latest_content_df(content_df):
    # Sort the entire DataFrame by "category" and "date" in descending order
    sorted_content_df = content_df.sort_values(
        by=["category", "date"], ascending=[True, False]
    )

    # Get the first row for each category using groupby and head
    latest_content_df = sorted_content_df.groupby("category").head(1).copy()
    latest_content_df["date"] = pd.to_datetime(
        latest_content_df["date"], format="%m-%d-%Y"
    )
    latest_content_df = latest_content_df.sort_values(by="date", ascending=False)

    return latest_content_df

##### Homepage

Storing selected columns for Homepage only

In [461]:
# Needed columns for homepage
columns_initial = [
    "article_id",
    "category",
    "date",
    "tags",
    "title",
    "cover_image",
    "short_description",
]
content_df = load_content_from_files(columns_initial)
latest_content_df = get_latest_content_df(content_df)

In [462]:
latest_content_df

,article_id,category,date,tags,title,cover_image,short_description
3,snr0509_josh_paper,Astrophysical Transients,NaT,"[research, news]",A comprehensive SN Ia companion search in SNR ...,website_files/images/article_content/snr0509_v...,A search for a surviving companion to a 400 ye...
0,uuraf21_poster_vicente,Computational Metascience,NaT,[research],MSU UURAF 2021,website_files/images/article_content/MAST_Post...,Poster presentation for MSU's University Under...
1,news_nsf22_collab,News,NaT,"[news, research, grant]",Collaborative Research: Unravelling Stripped S...,website_files/images/article_content/nsf22_col...,We are thrilled to announce that our NSF resea...


In [463]:
# Rendering the homepage template with data
homepage_template = environment.get_template("homepage.html.j2")
homepage_content = homepage_template.render(
    general=data["general"],
    homepage=data["homepage"],
    recent_content=latest_content_df.to_dict(orient="records"),
)
homepage_file_path = HOSTING_PATH / "Index.html"

with open(homepage_file_path, mode="w", encoding="utf-8") as homepage:
    homepage.write(homepage_content)

##### People Page

In [464]:
# Function to parse member data from JSON files
def parse_member_data(member_dir):
    member_json_dir = member_dir / "jsons"
    education_experience_list = []
    file_names = ["experiences.json", "education.json"]

    for file_name in file_names:
        file_path = member_json_dir / file_name
        if file_path.exists():
            with open(file_path, "r") as f:
                education_experience_list += json.load(f)
        else:
            print(file_path)

    # Load social links directly
    social_links = {}
    social_links_file_path = member_json_dir / "social_links.json"
    if social_links_file_path.exists():
        with open(social_links_file_path, "r") as f:
            social_links = json.load(f)

    # Load topmost project title
    current_project_title = None
    projects_file_path = member_json_dir / "projects.json"
    if projects_file_path.exists():
        with open(projects_file_path, "r") as f:
            projects = json.load(f)
            if projects:  # ensure it's not an empty list
                current_project_title = projects[0].get("project_title")

    return pd.DataFrame(education_experience_list), social_links, current_project_title

In [465]:
# member_dir = Path("/Users/harshul/projects/kgwebsite/group-data/members/wolfgang_kerzendorf")
# parse_member_data(member_dir)

In [466]:
# Function to extract academic roles from education and experience data


def extract_member_academic_role(education_experience_df):
    # Check if these columns exist in dataframe
    for column in ["end_date", "group", "institution"]:
        if column not in education_experience_df.columns:
            education_experience_df[column] = None

    current_academic_role = None

    role_map = {
        "Assistant Professor": "Professor",
        "Professor": "Professor",
        "Visualization Consultant": "Visualization Consultant",
        "Research Consultant": "Research Consultant",
        "Research Software Engineer": "Research Software Engineer",
        "Professorial Assistant": "Undergraduate",
        "Visiting Researcher": "Postdoctoral Researcher",
        "Postdoctoral Researcher": "Postdoctoral Researcher",
    }

    degree_map = {
        "Masters": "Graduate Student",
        "PhD": "Postdoctorate",  #  if end_date is present
        "Bachelors": "Graduate Student",
    }

    for _, row in education_experience_df.iterrows():
        role = row.get("role", None)
        degree = row.get("degree", None)

        if not current_academic_role:
            current_academic_role = role_map.get(role, "")

            if degree == "PhD" and pd.isna(row["end_date"]):
                current_academic_role = "Graduate Student"  # if end_date is NaN
            elif degree == "Bachelors" and pd.isna(row["end_date"]):
                current_academic_role = "Undergraduate Student"
            elif not current_academic_role and degree in degree_map:
                current_academic_role = degree_map[degree]

    # Check for end dates outside the loop
    has_end_date = all(
        not pd.isna(date) for date in education_experience_df["end_date"]
    )
    is_current_member = not has_end_date

    return current_academic_role, is_current_member

In [467]:
# Lists to store data for current and alumni members

current_people_page_list = []
alumni_people_page_list = []

# Looping through member directories to fetch and process member data
for member_dir in MEMBERS_DIR_PATH.glob("*"):
    print(member_dir)
    if not (member_info_fname := member_dir / "info.json").exists():
        continue
    else:
        member_info = json.load(open(member_info_fname, "r"))
    education_experience_df, social_links, current_project_title = parse_member_data(
        member_dir
    )
    current_academic_role, is_current_member = extract_member_academic_role(
        education_experience_df
    )

    first_name = member_info["first_name"]
    # print(first_name)
    last_name = member_info["last_name"]
    nickname = member_info.get("nick_name", None)
    id = member_info["id"]
    image_path = member_info["image_path"]
    cover_image_path = member_info["cover_image_path"]

    name = f"{nickname if nickname else first_name} {last_name}"

    member_data = {
        "name": name,
        "academic_role": current_academic_role,
        "id": id,
        "current_project_title": current_project_title,
        "image_path": image_path,
        "cover_image_path": cover_image_path,
    }

    member_data.update(social_links)

    if is_current_member:
        current_people_page_list.append(member_data)
    else:
        alumni_people_page_list.append(member_data)

/Users/harshul/projects/kgwebsite/group-data/members/josh_shields
/Users/harshul/projects/kgwebsite/group-data/members/anirban_dutta
/Users/harshul/projects/kgwebsite/group-data/members/erin_visser
/Users/harshul/projects/kgwebsite/group-data/members/erin_visser/jsons/experiences.json
/Users/harshul/projects/kgwebsite/group-data/members/.DS_Store
/Users/harshul/projects/kgwebsite/group-data/members/vicente_amado
/Users/harshul/projects/kgwebsite/group-data/members/vicente_amado/jsons/experiences.json
/Users/harshul/projects/kgwebsite/group-data/members/yuki_matsumura
/Users/harshul/projects/kgwebsite/group-data/members/yuki_matsumura/jsons/experiences.json
/Users/harshul/projects/kgwebsite/group-data/members/andrew_fullard
/Users/harshul/projects/kgwebsite/group-data/members/isaac_smith
/Users/harshul/projects/kgwebsite/group-data/members/hayden_monk
/Users/harshul/projects/kgwebsite/group-data/members/hayden_monk/jsons/experiences.json
/Users/harshul/projects/kgwebsite/group-data/memb

In [468]:
# current_people_page_list
# alumni_people_page_list

In [469]:
# Rendering the people page template with data and saving it to a file
people_template = environment.get_template("people.html.j2")
# Passing lists to jinja2 template
people_content = people_template.render(
    general=data["general"],
    current_members=current_people_page_list,
    alumni_members=alumni_people_page_list,
)
people_file_path = HOSTING_PATH / "People.html"

with open(people_file_path, mode="w", encoding="utf-8") as people:
    people.write(people_content)

For adding more columns in dataframe to render fronnt pages and individual article pages

In [470]:
columns_extended = columns_initial + ["author_id", "content"]
content_df = load_content_from_files(columns_extended)

latest_content_df = get_latest_content_df(content_df)
latest_content_df

,article_id,category,date,tags,title,cover_image,short_description,author_id,content
3,snr0509_josh_paper,Astrophysical Transients,NaT,"[research, news]",A comprehensive SN Ia companion search in SNR ...,website_files/images/article_content/snr0509_v...,A search for a surviving companion to a 400 ye...,josh_shields,{'para1': 'An example outcome of the fitting p...
0,uuraf21_poster_vicente,Computational Metascience,NaT,[research],MSU UURAF 2021,website_files/images/article_content/MAST_Post...,Poster presentation for MSU's University Under...,vicente_amado,{'para1': 'Abstract: The modern scientific com...
1,news_nsf22_collab,News,NaT,"[news, research, grant]",Collaborative Research: Unravelling Stripped S...,website_files/images/article_content/nsf22_col...,We are thrilled to announce that our NSF resea...,wolfgang_kerzendorf,{}
